# Microfacet Distributions

Some definitions:

- $\alpha = \text{roughness}^2 \in [0, 1]$: we remap `roughness` $\in [0, 1]$ to $\alpha$, similar to [Bur2012] and [Kar2013]
- $\omega_n$: normal vector of macroscopic surface
- $\omega_v$: view vector, from surface to eye
- $\omega_l$: light vector, from surface to light
- $\omega_h = \frac{\omega_v + \omega_l}{\|\omega_v + \omega_l\|} = (h_x, h_y, h_z)$: half vector, normal vector of microfacet
- $\omega_x, \omega_y$: two tangent vectors of macroscopic surface so that $\omega_x, \omega_y, \omega_n$ form an orthonormal basis and that $\begin{cases}
  h_x = \omega_h \cdot \omega_x \\
  h_y = \omega_h \cdot \omega_y \\
  h_z = \omega_h \cdot \omega_n
  \end{cases}$
- $\theta_h$ and $\phi_h$ are the angles between $\omega_n$ and $\omega_h$, and $\omega_x$ and $\omega_h$; so that  $\begin{cases}
  h_x = \omega_h \cdot \omega_x = \cos \phi_h \sin \theta_h\\
  h_y = \omega_h \cdot \omega_y = \sin \phi_h \sin \theta_h\\
  h_z = \omega_h \cdot \omega_n = \cos \theta_h
  \end{cases}$

References:

- [AS2000] M. Ashikhmin, P. Shirley, An Anisotropic Phong BRDF Model (2000)
- [Bur2012] B. Burley, Physically Based Shading at Disney (2012)
Refraction through Rough Surfaces (2007)
- [Cao2015] [J. Cao, Sampling Microfacet BRDF (2015)](https://agraphicsguynotes.com/posts/sample_microfacet_brdf/) ([archived](https://web.archive.org/web/20231127203110/https://agraphicsguynotes.com/posts/sample_microfacet_brdf/))
- [Cao2018] [J. Cao, Sampling Anisotropic Microfacet BRDF](https://agraphicsguynotes.com/posts/sample_anisotropic_microfacet_brdf/) ([archived](https://web.archive.org/web/20210508135108/https://agraphicsguynotes.com/posts/sample_anisotropic_microfacet_brdf/))
- [Heitz2014] E. Heitz, Understanding the masking-shadowing function in microfacet-based BRDFs (2014)
- [Kar2013] [B. Karis, Specular BRDF Reference (2013)](http://graphicrants.blogspot.com/2013/08/specular-brdf-reference.html) ([archived](https://web.archive.org/web/20240110220254/http://graphicrants.blogspot.com/2013/08/specular-brdf-reference.html))
- [Neu1999] Neumann et al., Compact metallic reflectance models (1999)
- [WLMT2007] B. Walter, S.R. Marschner, H. Li, K.E. Torrance, Microfacet Model for 


## Normal Distribution Functions (NDF)

Each NDF $D(\omega_h)$ has a normalization constant $c$. For a plausible NDF we must have that:

$$\int_\mathcal{H^+} D(\omega_h)\cos\theta_h \mathrm{d}\omega_h = \int_0^{2\pi} \int_0^{\pi/2} D(\omega_h) \cos\theta_h \sin\theta_h \mathrm{d}\theta_h \mathrm{d}\phi_h = 1$$

From this, $c$ can be determined, but not all NDFs will have a closed form formula. But for the ones that do, it's often $c=\frac{1}{\pi \alpha^2}$

We can use this NDF for importance sampling of $\omega_h$ where:

$$\begin{align*}\mathrm{Pr}[\omega_h \in \Omega] 
    &= \int_\Omega \mathrm{p}(\omega_h) \mathrm{d}\omega_h \\
    &= \int_\Omega D(\omega_h)\cos\theta_h \mathrm{d}\omega_h \\
    &= \iint_\Omega D(\omega_h)\cos\theta_h \sin\theta_h \mathrm{d}\theta_h \mathrm{d}\phi_h
\end{align*}$$

This explains the factor $\cos\theta_h$ in $\mathrm{p}(\omega_h)$:

$$\mathrm{p}(\omega_h) = D(\omega_h)\cos\theta_h$$

And in spherical coordinates with $\mathrm{d}\omega_h = \sin\theta_h \mathrm{d}\theta_h \mathrm{d}\phi_h$:

$$\begin{align*}
\mathrm{p}(\theta_h, \phi_h) 
    &= \mathrm{p}(\omega_h) \sin\theta_h \\
    &= D(\omega_h) \cos\theta_h \sin\theta_h
\end{align*}$$

### Isotropic

In the isotropic cases this is separable, and we have $\mathrm{p}(\theta_h, \phi_h) = \mathrm{p}_\theta(\theta_h) \mathrm{p}_\phi(\phi_h)$, so that:

$$\begin{align*}
\mathrm{p}_\phi(\phi_h)   &= \frac{1}{2 \pi}\\
\mathrm{p}_\theta(\theta_h) &= 2 \pi D(\omega_h)\cos\theta_h \sin\theta_h
\end{align*}$$

We integrate these to get the CDFs:

$$\begin{align*}
\mathrm{P}_\phi(\phi_h) &= \int_0^{\phi_h} \frac{1}{2 \pi} \mathrm{d}_\phi = \frac{\phi_h}{2 \pi}\\
\mathrm{P}_\theta(\theta_h) &= \int_0^{\theta_h} 2 \pi D(\omega_h)\cos\theta_h \sin\theta_h \mathrm{d}_\theta
\end{align*}$$

Inverting these will yield equations to transform uniform random variables $(\xi_1, \xi_2) \in [0, 1)^2$ to $\phi_h$ and $\theta_h$:

$$\begin{align*}
\phi_h &= \mathrm{P}_\phi^{-1}(\xi_1) = 2 \pi \xi_1\\
\theta_h &= \mathrm{P}_\theta^{-1}(\xi_2) = \dots
\end{align*}$$

### Anisotropic

In this case, we'll split the PDF in a marginal PDF $\mathrm{p}_\phi(\phi_h)$ and conditional PDF $\mathrm{p}_{\theta|\phi}(\theta, \phi_h)$ so that:

$$\mathrm{p}(\theta_h, \phi_h) = \mathrm{p}_{\theta|\phi}(\theta, \phi_h) \mathrm{p}_\phi(\phi_h)$$

The marginal PDF $\mathrm{p}_\phi(\phi_h)$ is determined by integrating over $\theta$, and the conditional PDF $\mathrm{p}_{\theta|\phi}(\theta, \phi_h)$ is found by above equation:

$$\begin{align*}
\mathrm{p}_\phi(\phi_h) &= \int_0^\frac{\pi}{2} \mathrm{p}(\theta_h, \phi_h) \mathrm{d} \theta_h \\
\mathrm{p}_{\theta|\phi}(\theta, \phi_h) &= \frac{ \mathrm{p}(\theta_h, \phi_h) } { \mathrm{p}_\phi(\phi_h) } \\
\end{align*}$$

We determine CDFs of both:

$$\begin{align*}
\mathrm{P}_\phi(\phi_h) &= \int_0^{\phi_h} \mathrm{p}_\phi(\phi) \mathrm{d}_\phi\\
\mathrm{P}_{\theta|\phi}(\theta_h, \phi_h) &= \int_0^{\theta_h} \mathrm{p}_{\theta|\phi}(\theta, \phi_h) \mathrm{d}_\theta
\end{align*}$$

Inverting these will yield equations to transform uniform random variables $(\xi_1, \xi_2) \in [0, 1)^2$ to $\phi_h$ and $\theta_h$:

$$\begin{align*}
\phi_h &= \mathrm{P}_\phi^{-1}(\xi_1) = \dots\\
\theta_h &= \mathrm{P}_{\theta|\phi}^{-1}(\xi_2, \phi_h) = \dots
\end{align*}$$

### Blinn-Phong

#### Isotropic

With $n \in [0, \infty]$ the specular power or shininess:

$$D_\text{Blinn}(\omega_h) = c (\cos\theta_h)^n$$

Let's normalize:

$$\begin{align*}
\int_\mathcal{H^+} D(\omega_h)\cos\theta_h \mathrm{d}\omega_h 
    &= \int_0^{2\pi} \int_0^{\pi/2} c (\cos\theta_h)^n \cos\theta_h \sin\theta_h \mathrm{d}\theta_h \mathrm{d}\phi_h \\
    &= 2 \pi c \int_0^{\pi/2} (\cos\theta_h)^{n+1} \sin\theta_h \mathrm{d}\theta_h \\
    &\qquad \text{substitute} \left\{ \begin{align*}
        u &= \cos\theta_h \\
        \mathrm{d}u &= -\sin \theta_h \mathrm{d}\theta_h
    \end{align*} \right.\\
    &= 2 \pi c \int_1^0 -u^{n+1} \mathrm{d}u \\
    &= 2 \pi c \left[ \frac {-u^{n+2}}{n + 2} \right]_1^0 \\
    &= \frac {2 \pi c}{n + 2} \\
    &= 1 \\
    &\Downarrow \\
  c &= \frac{n+2}{2\pi}
\end{align*}
$$

If we substitute $n = \frac{2}{\alpha^2}-2$, we get $c = \frac{1}{\pi \alpha^2}$ and:

$$D_\text{Blinn}(\omega_h) = \frac{1}{\pi \alpha^2}(\cos\theta_h)^{\frac{2}{\alpha^2} - 2}$$

For importance sampling, we want to sample $\theta_h$ according to $\mathrm{p}_\theta(\theta_h)$:

$$\begin{align*}
\mathrm{p}_\theta(\theta_h)
    &= 2 \pi D(\omega_h)\cos\theta_h \sin\theta_h \\
    &= \frac{2}{\alpha^2} (\cos\theta_h)^{\frac{2}{\alpha^2} - 1} \sin\theta_h
\end{align*}$$

For this we need to CDF $\mathrm{P}_\theta(\theta_h)$:

$$\begin{align*}
\mathrm{P}_\theta(\theta_h)
    &= \int_0^{\theta_h} \mathrm{p}_\theta(\theta) \mathrm{d}\theta \\
    &= \frac{2}{\alpha^2} \int_0^{\theta_h} (\cos\theta)^{\frac{2}{\alpha^2} - 1} \sin\theta \mathrm{d}\theta \\
    &\qquad \text{substitute} \left\{ \begin{align*}
        u &= \cos\theta \\
        \mathrm{d}u &= -\sin\theta \mathrm{d}\theta
    \end{align*} \right.\\
    &= -\frac{2}{\alpha^2} \int_1^{\cos\theta_h} u^{\frac{2}{\alpha^2} - 1} \mathrm{d}u \\
    &= -\frac{2}{\alpha^2} \Big[\frac{u^{\frac{2}{\alpha^2}}}{\frac{2}{\alpha^2}}\Big]_1^{\cos\theta_h} \\
    &= 1 - \left(\cos\theta_h\right)^{\frac{2}{\alpha^2}} \\
    &= 1 - \left(\cos\theta_h\right)^{n+2}
\end{align*}$$

Inverting this function and solving for $\theta_h$ we get:

$$\begin{align*}
\phi_h   &= 2 \pi \xi_1 \\
\theta_h &= \arccos \left(  \left( 1 - \xi_2 \right)^{\frac{\alpha^2}{2}} \right) \\
         &= \arccos \left(  \left( 1 - \xi_2 \right)^{\frac{1}{n+2}} \right)
\end{align*}$$

#### Anistropic

For the anistropic distribution, roughness is varied by $\phi$ by substituting:

$$\frac{1}{\alpha^2}=\frac{\cos^2\phi}{\alpha_x^2}+\frac{\sin^2\phi}{\alpha_y^2}$$

With $\alpha^2 = \frac{2}{n+2}$ this becomes 

$$\begin{align*}
n &= \frac{2}{\alpha^2} - 2 \\
  &= 2 \left( \frac{\cos^2\phi}{\alpha_x^2}+\frac{\sin^2\phi}{\alpha_y^2} \right) - 2 \left( \cos^2\phi + \sin^2\phi \right) \\
  &= \left( \frac{2}{\alpha_x^2} - 2 \right) \cos^2\phi + \left( \frac{2}{\alpha_y^2} - 2 \right) \sin^2\phi \\
  &= n_x \cos^2\phi + n_y \sin^2\phi
\end{align*}$$

$$\begin{align*}
D_\text{Blinn,aniso}(\omega_h) 
  &= \frac{1}{\pi \alpha_x \alpha_y}(\cos\theta_h)^{2(\frac{\cos^2\phi}{\alpha^2_x}+\frac{\sin^2\phi}{\alpha^2_y}) - 2} \\
  &= \frac{\sqrt{(n_x+2)(n_y+2)}}{2\pi}(\cos\theta_h)^{n_x \cos^2\phi + n_y \sin^2\phi}
\end{align*}$$

This has a non-separable PDF:

$$\begin{align*}
\mathrm{p}(\theta_h, \phi_h)
    &= D_\text{Blinn,aniso}(\omega_h) \cos \theta_h \sin \theta_h \\
    &= \frac{\sin \theta_h}{\pi \alpha_x \alpha_y}(\cos\theta_h)^{2(\frac{\cos^2\phi}{\alpha^2_x}+\frac{\sin^2\phi}{\alpha^2_y}) - 1} \\
\end{align*}$$

Let's first determine the marginal PDF for $\phi_h$.

$$\begin{align*}
\mathrm{p}_\phi(\phi_h)
    &= \int_0^{\frac{\pi}{2}} \mathrm{p}(\theta_h, \phi_h) \mathrm{d}\theta_h \\
    &= \int_0^{\frac{\pi}{2}} \frac{\sin \theta_h}{\pi \alpha_x \alpha_y}(\cos\theta_h)^{2(\frac{\cos^2\phi}{\alpha^2_x}+\frac{\sin^2\phi}{\alpha^2_y}) - 1} \mathrm{d}\theta_h \\
    &\qquad \text{substitute} \left\{ \begin{align*}
        A(\phi_h) &= \frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y} > 0\\
        u &= \cos\theta \\
        \mathrm{d}u &= -\sin\theta \mathrm{d}\theta
    \end{align*} \right.\\
    &= -\frac{1}{\pi \alpha_x \alpha_y} \int_1^0 u^{2A(\phi_h)-1} \mathrm{d}u \\
    &= -\frac{1}{\pi \alpha_x \alpha_y} \Bigg[\frac{u^{2A(\phi_h)}}{2A(\phi_h)} \Bigg]_1^0 \\
    &= \frac{1}{2 \pi \alpha_x \alpha_y A(\phi_h) } \\
    &= \frac{1}{2 \pi \alpha_x \alpha_y \left( \frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y} \right) }
\end{align*}$$

Let's integrate this to get the marginal CDF for $\phi_h$:

$$\begin{align*}
\mathrm{P}_\phi(\phi_h)
    &= \int_0^{\phi_h} \mathrm{p}_\phi(\phi_h) \mathrm{d}\phi \\
    &= \frac{1}{2 \pi \alpha_x \alpha_y} \int_0^{\phi_h} \frac{1}{ \frac{\cos^2\phi}{\alpha^2_x}+\frac{\sin^2\phi}{\alpha^2_y} } \mathrm{d}\phi \\
    &= \frac{\alpha_x}{2 \pi \alpha_y} \int_0^{\phi_h} \frac{1}{ \left( 1+\frac{\alpha^2_x \tan^2\phi}{\alpha^2_y} \right) \cos^2\phi} \mathrm{d}\phi \\
    &\qquad \text{substitute} \left\{ \begin{align*}
        u &= \frac{\alpha_x}{\alpha_y} \tan \phi \\
        \mathrm{d}u &= \frac{\alpha_x}{\alpha_y} \frac{1}{\cos^2\phi} \mathrm{d}\phi
    \end{align*} \right.\\
    &= \frac{1}{2 \pi} \int_0^{\frac{\alpha_x}{\alpha_y} \tan \phi_h} \frac{1}{1 + u^2} \mathrm{d}u \\
    &= \frac{1}{2 \pi} \Big[\arctan u \Big]_0^{\frac{\alpha_x}{\alpha_y} \tan \phi_h} \\
    &= \frac{1}{2 \pi} \arctan \left( \frac{\alpha_x}{\alpha_y} \tan \phi_h \right) \\
\end{align*}$$

Inverting this function and solving for $\phi_h$ we get:

$$\begin{align*}
\phi_h 
    &= \mathrm{P}_\phi^{-1}(\xi_1) \\
    &= \arctan \left( \frac{\alpha_y}{\alpha_x} \tan \left( 2 \pi \xi_1 \right) \right)
\end{align*}$$

Care must be taking to evaluate this. Both [AS2000] and [Cao2018] describe mappings, both evaluate to the same curve (see graph below), but the latter is simpler

$$\phi_h = \begin{cases}
    \arctan \left( \frac{\alpha_y}{\alpha_x} \tan \left( 2 \pi \xi_1 \right) \right) & \text{if } \xi_1 \leq 0.25 \\
    \arctan \left( \frac{\alpha_y}{\alpha_x} \tan \left( 2 \pi \xi_1 \right) \right) + \pi & \text{if } 0.25 < \xi_1 < 0.75 \\
    \arctan \left( \frac{\alpha_y}{\alpha_x} \tan \left( 2 \pi \xi_1 \right) \right) + 2 \pi & \text{if } \xi_1 \geq 0.75 \\
\end{cases}$$

Now we can work out the conditional PDF for $\theta_h$:

$$\begin{align*}
\mathrm{p}_{\theta|\phi}(\theta_h, \phi_h)
    &= \frac{\mathrm{p}(\theta_h, \phi_h)}{\mathrm{p}_\phi(\phi_h)} \\
    &= 2 A(\phi_h) \sin \theta_h (\cos\theta_h)^{2A(\phi_h) - 1} \\
\end{align*}$$

Again, integrate to the the conditional CDF for $\theta_h$:

$$\begin{align*}
\mathrm{P}_{\theta|\phi}(\theta_h, \phi_h)
    &= 2 A(\phi_h) \int_0^{\theta_h} \sin \theta (\cos\theta)^{2A(\phi_h) - 1} \mathrm{d}\theta \\
    &\qquad \text{substitute} \left\{ \begin{align*}
        u &= \cos \theta \\
        \mathrm{d}u &= -\sin \theta \mathrm{d}\theta
    \end{align*} \right.\\
    &=  -2 A(\phi_h) \int_1^{\cos \theta_h} u^{2A(\phi_h) - 1} \mathrm{d}u \\
    &=  -2 A(\phi_h) \Bigg[\frac{u^{2A(\phi_h)}}{2A(\phi_h)}\Bigg]_1^{\cos \theta_h} \\
    &= 1 - \left(\cos \theta_h\right)^{2A(\phi_h)}
\end{align*}$$

Inverting this function and solving for $\theta_h$ we get:

$$\begin{align*}
\theta_h 
    &= \mathrm{P}_{\theta|\phi}^{-1}(\xi_2, \phi_h) \\
    &= \arccos \left(\left( 1 - \xi_2 \right)^{\frac{1}{2A(\phi_h)}} \right) \\
    &= \arccos \left(\left( 1 - \xi_2 \right)^{\frac{1}{2 \left( \frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y} \right)}} \right) \\
    &= \arccos \left(\left( 1 - \xi_2 \right)^{\frac{1}{n_x \cos^2\phi + n_y \sin^2\phi + 2}} \right)
\end{align*}$$

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np


def sample_phi_AS2000(alpha_x, alpha_y, xi):
    """As described in [AS2000]"""
    a = alpha_y / alpha_x
    if xi < 0.25:
        return math.atan(a * math.tan(2 * math.pi * xi))
    elif xi < 0.5:
        return math.pi - math.atan(a * math.tan(2 * math.pi * (0.5 - xi)))
    elif xi < 0.75:
        return math.pi + math.atan(a * math.tan(2 * math.pi * (xi - 0.5)))
    else:
        return 2 * math.pi - math.atan(a * math.tan(2 * math.pi * (1 - xi)))


def sample_phi_Cao2018(alpha_x, alpha_y, xi):
    """As described in [Cao2018]"""
    a = alpha_y / alpha_x
    if xi <= 0.25:
        return math.atan(a * math.tan(2 * math.pi * xi))
    elif xi < 0.75:
        return math.pi + math.atan(a * math.tan(2 * math.pi * xi))
    else:
        return 2 * math.pi + math.atan(a * math.tan(2 * math.pi * xi))


def sample_phi(alpha_x, alpha_y, xi):
    a = alpha_y / alpha_x
    phi = math.atan(a * math.tan(2 * math.pi * xi + math.tan(2 * math.pi / 2)))
    if xi > 0.5:
        phi += math.pi
    return phi


alpha_x = 0.1
alpha_y = 0.2
xi = np.arange(0, 1, 0.001, dtype=np.float32)

plt.plot(
    xi,
    [sample_phi_AS2000(alpha_x, alpha_y, x) for x in xi],
    label="AS2000",
    color="red",
)
plt.plot(
    xi,
    [sample_phi_Cao2018(alpha_x, alpha_y, x) for x in xi],
    label="Cao2018",
    color="blue",
    linestyle="dashed",
)
plt.plot(
    xi,
    [sample_phi(alpha_x, alpha_y, x) for x in xi],
    label="sample_phi",
    color="green",
    linestyle="dotted",
)
plt.xlabel(r"$\xi_1$")
plt.ylabel(r"$\phi_h$")
plt.legend()
plt.show()

### Beckmann

#### Isotropic

$$D_\text{Beckmann}(\omega_h) = \frac{1}{\pi \alpha^2 \cos^4\theta_h}\exp \left( -\frac{\tan^2\theta_h}{\alpha^2} \right)$$

For importance sampling, we want to sample $\theta_h$ according to $\mathrm{p}_\theta(\theta_h)$:

$$\begin{align*}
\mathrm{p}_\theta(\theta_h)
    &= 2 \pi D(\omega_h)\cos\theta_h \sin\theta_h \\
    &= \frac{2 \sin\theta_h}{\alpha^2 \cos^3\theta_h}\exp \left( -\frac{\tan^2\theta_h}{\alpha^2} \right)
\end{align*}$$

For this we need to CDF $\mathrm{P}_\theta(\theta_h)$:

$$\begin{align*}
\mathrm{P}_\theta(\theta_h)
    &= \int_0^{\theta_h} \frac{2 \sin\theta}{\alpha^2 \cos^3\theta}\exp \left( -\frac{\tan^2\theta}{\alpha^2} \right) \mathrm{d}\theta \\
    &\qquad \text{substitute} \left\{ \begin{align*}
        u &= -\frac{\tan^2\theta}{\alpha^2} \\
        \mathrm{d}u &= -\frac{2\tan\theta}{\alpha^2 \cos^2\theta} \mathrm{d}\theta = -\frac{2\sin\theta}{\alpha^2 \cos^3\theta} \mathrm{d}\theta
    \end{align*} \right.\\
    &= -\int_0^{-\frac{\tan^2\theta_h}{\alpha^2}} \exp(u) \mathrm{d}u \\
    &= -\Big[ \exp(u) \Big]_0^{-\frac{\tan^2\theta_h}{\alpha^2}} \\
    &= 1 - \exp \left( -\frac{\tan^2\theta_h}{\alpha^2} \right)
\end{align*}$$

Inverting this function and solving for $\theta_h$ we get:

$$\begin{align*}
\phi_h   &= 2 \pi \xi_1 \\
\theta_h &= \arctan \left( \alpha \sqrt { - \ln \left( 1 - \xi_2 \right) } \right)
\end{align*}$$


#### Anistropic

For the anistropic distribution, roughness is varied by $\phi_h$ by substituting $\frac{1}{\alpha^2}=\frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y}$

$$\begin{align*}
D_\text{Beckmann,aniso}(\omega_h)
    &= \frac{1}{\pi \alpha_x \alpha_y \cos^4\theta_h} \exp \left( -\tan^2\theta_h \left(\frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y} \right) \right) \\
    &= \frac{1}{\pi \alpha_x \alpha_y h_z^4} \exp \left( -\frac{1}{h_z^2} \left( \frac{h_x^2}{\alpha^2_x}+\frac{h_y^2}{\alpha^2_y} \right) \right)
\end{align*}$$

This has a non-separable PDF:

$$\begin{align*}
\mathrm{p}(\theta_h, \phi_h)
    &= \frac{\sin\theta_h}{\pi \alpha_x \alpha_y \cos^3\theta_h} \exp \left( -\tan^2\theta_h \left(\frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y} \right) \right) \\
\end{align*}$$

Let's first determine the marginal PDF for $\phi_h$.

$$\begin{align*}
\mathrm{p}_\phi(\phi_h)
    &= \int_0^{\frac{\pi}{2}} \mathrm{p}(\theta_h, \phi_h) \mathrm{d}\theta_h \\
    &= \int_0^{\frac{\pi}{2}} \frac{\sin\theta_h}{\pi \alpha_x \alpha_y \cos^3\theta_h} \exp \left( -\tan^2\theta_h \left(\frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y} \right) \right) \mathrm{d}\theta_h \\
    &\qquad \text{substitute} \left\{ \begin{align*}
        A(\phi_h) &= \frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y} > 0\\
        u &= -\tan^2\theta A(\phi_h) \\
        \mathrm{d}u &= -\frac{2\tan\theta A(\phi_h)}{\cos^2\theta} \mathrm{d}\theta = -\frac{2\sin\theta A(\phi_h)}{\cos^3\theta} \mathrm{d}\theta
    \end{align*} \right.\\
    &= -\frac{1}{2 \pi \alpha_x \alpha_y A(\phi_h) } \int_0^{-\infty} \exp(u) \mathrm{d}u \\
    &= -\frac{1}{2 \pi \alpha_x \alpha_y A(\phi_h) } \Big[ \exp(u) \Big]_0^{-\infty} \\
    &= \frac{1}{2 \pi \alpha_x \alpha_y A(\phi_h) } \\
    &= \frac{1}{2 \pi \alpha_x \alpha_y \left( \frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y} \right) }
\end{align*}$$

This is the same as above, so that:

$$\phi_h = \begin{cases}
    \arctan \left( \frac{\alpha_y}{\alpha_x} \tan \left( 2 \pi \xi_1 \right) \right) & \text{if } \xi_1 \leq 0.25 \\
    \arctan \left( \frac{\alpha_y}{\alpha_x} \tan \left( 2 \pi \xi_1 \right) \right) + \pi & \text{if } 0.25 < \xi_1 < 0.75 \\
    \arctan \left( \frac{\alpha_y}{\alpha_x} \tan \left( 2 \pi \xi_1 \right) \right) + 2 \pi & \text{if } \xi_1 \geq 0.75 \\
\end{cases}$$

Now we can work out the conditional PDF for $\theta_h$:

$$\begin{align*}
\mathrm{p}_{\theta|\phi}(\theta_h, \phi_h)
    &= \frac{\mathrm{p}(\theta_h, \phi_h)}{\mathrm{p}_\phi(\phi_h)} \\
    &= \frac{2 A(\phi_h) \sin\theta_h}{\cos^3\theta_h} \exp \left( -\tan^2\theta_h A(\phi_h) \right) \\
\end{align*}$$

Again, integrate to the the conditional CDF for $\theta_h$:

$$\begin{align*}
\mathrm{P}_{\theta|\phi}(\theta_h, \phi_h)
    &= \int_0^{\theta_h} \frac{2 A(\phi_h) \sin\theta}{\cos^3\theta} \exp \left( -\tan^2\theta_h A(\phi_h) \right) \mathrm{d}\theta \\
    &\qquad \text{substitute} \left\{ \begin{align*}
        u &= -\tan^2\theta A(\phi_h) \\
        \mathrm{d}u &= -\frac{2\sin\theta A(\phi_h)}{\cos^3\theta} \mathrm{d}\theta
    \end{align*} \right.\\
    &= - \int_0^{-\tan^2\theta_h A(\phi_h)} \exp (u) \mathrm{d}u \\
    &=  1 - \exp \left( -\tan^2\theta_h A(\phi_h) \right) \\
\end{align*}$$

Inverting this function and solving for $\theta_h$ we get:

$$\begin{align*}
\theta_h 
    &= \mathrm{P}_{\theta|\phi}^{-1}(\xi_2, \phi_h) \\
    &= \arctan \sqrt{-\frac{\ln \left(1 - \xi_2 \right)}{A(\phi_h)}} \\
    &= \arctan \sqrt{-\frac{\ln \left(1 - \xi_2 \right)}{\frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y}}} \\
\end{align*}$$

### Trowbridge-Reitz, GGX, GTR

### Isotropic

The Trowbridge-Reitz distribution was reinvented by [WLMT2007] where it was called GGX.

$$\begin{align*}
D_{\text{TR}}(\omega_h) 
    &= D_{\text{GGX}}(\omega_h) = D_\text{GTR2}(\omega_h) \\
    &= \frac{\alpha^2}{\pi} \left(\frac{1}{\sin^2\theta_h+\alpha^2 \cos^2\theta_h}\right)^2 \\
    &= \frac{\alpha^2}{\pi} \left(\frac{1}{1 + (\alpha^2 - 1) \cos^2\theta_h}\right)^2 \\
    &= \frac{1}{\pi \alpha^2 \cos^4\theta_h} \left(\frac{1}{\frac{\tan^2\theta_h}{\alpha^2} + 1}\right)^2
\end{align*}$$

For importance sampling, we want to sample $\theta_h$ according to $\mathrm{p}_\theta(\theta_h)$:

$$\begin{align*}
\mathrm{p}_\theta(\theta_h)
    &= 2 \pi D(\omega_h)\cos\theta_h \sin\theta_h \\
    &= \frac{2 \sin\theta_h}{\alpha^2 \cos^3\theta_h} \left(\frac{1}{\frac{\tan^2\theta_h}{\alpha^2} + 1}\right)^2
\end{align*}$$

For this we need to CDF $\mathrm{P}_\theta(\theta_h)$:

$$\begin{align*}
\mathrm{P}_\theta(\theta_h)
    &= \int_0^{\theta_h} \frac{2 \sin\theta}{\alpha^2 \cos^3\theta} \left(\frac{1}{\frac{\tan^2\theta}{\alpha^2} + 1}\right)^2 \mathrm{d}\theta \\
    &\qquad \text{substitute} \left\{ \begin{align*}
        u &= \frac{\tan^2\theta}{\alpha^2} + 1 \\
        \mathrm{d}u &= \frac{2\tan\theta}{\alpha^2 \cos^2\theta} \mathrm{d}\theta = \frac{2\sin\theta}{\alpha^2 \cos^3\theta} \mathrm{d}\theta
    \end{align*} \right.\\
    &= \int_1^{\frac{\tan^2\theta_h}{\alpha^2} + 1} \frac{1}{u^2} \mathrm{d}u \\
    &= \Big[ -\frac{1}{u} \Big]_1^{\frac{\tan^2\theta}{\alpha^2} + 1} \\
    &= 1 - \frac{1}{\frac{\tan^2\theta_h}{\alpha^2} + 1}
\end{align*}$$

Inverting this function and solving for $\theta_h$ we get:

$$\begin{align*}
\phi_h   &= 2 \pi \xi_1 \\
\theta_h &= \arctan \left( \alpha \sqrt { \frac{\xi_2}{1 - \xi_2} } \right)
\end{align*}$$

### Anisotropic

For the anistropic distribution, roughness is varied by $\phi$ by substituting $\frac{1}{\alpha^2}=\frac{\cos^2\phi}{\alpha^2_x}+\frac{\sin^2\phi}{\alpha^2_y}$

$$\begin{align*}
D_{\text{TR,aniso}}(\omega_h)
    &= \frac{1}{\pi \alpha_x \alpha_y} \left(\frac{1}{\sin^2\theta_h(\frac{\cos^2\phi}{\alpha^2_x}+\frac{\sin^2\phi}{\alpha^2_y}) + \cos^2\theta_h}\right)^2 \\
    &= \frac{1}{\pi \alpha_x \alpha_y} \left(\frac{1}{(\frac{h_x}{\alpha_x})^2+(\frac{h_y}{\alpha_y})^2 + h_z^2}\right)^2
\end{align*}$$


This has a non-separable PDF:

$$\begin{align*}
\mathrm{p}(\theta_h, \phi_h)
    &= \frac{\sin\theta_h}{\pi \alpha_x \alpha_y \cos^3\theta_h} \left(\frac{1}{\tan^2\theta_h \left(\frac{\cos^2\phi}{\alpha^2_x}+\frac{\sin^2\phi}{\alpha^2_y}\right) + 1}\right)^2 \\
\end{align*}$$

Let's first determine the marginal PDF for $\phi_h$.

$$\begin{align*}
\mathrm{p}_\phi(\phi_h)
    &= \int_0^{\frac{\pi}{2}} \mathrm{p}(\theta_h, \phi_h) \mathrm{d}\theta_h \\
    &= \int_0^{\frac{\pi}{2}} \frac{\sin\theta_h}{\pi \alpha_x \alpha_y \cos^3\theta_h} \left(\frac{1}{\tan^2\theta_h \left(\frac{\cos^2\phi}{\alpha^2_x}+\frac{\sin^2\phi}{\alpha^2_y}\right) + 1}\right)^2 \mathrm{d}\theta_h \\
    &\qquad \text{substitute} \left\{ \begin{align*}
        A(\phi_h) &= \frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y} > 0\\
        u &= \tan^2\theta A(\phi_h) + 1 \\
        \mathrm{d}u &= \frac{2 \tan\theta A(\phi_h)}{\cos^2\theta} \mathrm{d}\theta = \frac{2 \sin\theta A(\phi_h)}{\cos^3\theta} \mathrm{d}\theta
    \end{align*} \right.\\
    &= \frac{1}{2 \pi \alpha_x \alpha_y A(\phi_h) } \int_1^{\infty} \frac{1}{u^2} \mathrm{d}u \\
    &= \frac{1}{2 \pi \alpha_x \alpha_y A(\phi_h) } \Big[ -\frac{1}{u} \Big]_1^{\infty} \\
    &= \frac{1}{2 \pi \alpha_x \alpha_y A(\phi_h) } \\
    &= \frac{1}{2 \pi \alpha_x \alpha_y \left( \frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y} \right) }
\end{align*}$$

This is the same as above, so that:

$$\phi_h = \begin{cases}
    \arctan \left( \frac{\alpha_y}{\alpha_x} \tan \left( 2 \pi \xi_1 \right) \right) & \text{if } \xi_1 \leq 0.25 \\
    \arctan \left( \frac{\alpha_y}{\alpha_x} \tan \left( 2 \pi \xi_1 \right) \right) + \pi & \text{if } 0.25 < \xi_1 < 0.75 \\
    \arctan \left( \frac{\alpha_y}{\alpha_x} \tan \left( 2 \pi \xi_1 \right) \right) + 2 \pi & \text{if } \xi_1 \geq 0.75 \\
\end{cases}$$


Now we can work out the conditional PDF for $\theta_h$:

$$\begin{align*}
\mathrm{p}_{\theta|\phi}(\theta_h, \phi_h)
    &= \frac{\mathrm{p}(\theta_h, \phi_h)}{\mathrm{p}_\phi(\phi_h)} \\
    &= \frac{2 A(\phi_h) \sin\theta_h}{\cos^3\theta_h} \left(\frac{1}{\tan^2\theta_h \left(\frac{\cos^2\phi}{\alpha^2_x}+\frac{\sin^2\phi}{\alpha^2_y}\right) + 1}\right)^2 \\
\end{align*}$$

Again, integrate to the the conditional CDF for $\theta_h$:

$$\begin{align*}
\mathrm{P}_{\theta|\phi}(\theta_h, \phi_h)
    &= \int_0^{\theta_h} \frac{2 A(\phi_h) \sin\theta}{\cos^3\theta} \left(\frac{1}{\tan^2\theta_h \left(\frac{\cos^2\phi}{\alpha^2_x}+\frac{\sin^2\phi}{\alpha^2_y}\right) + 1}\right)^2 \mathrm{d}\theta \\
    &\qquad \text{substitute} \left\{ \begin{align*}
u &= \tan^2\theta A(\phi_h) + 1 \\
        \mathrm{d}u &= \frac{2 \sin\theta A(\phi_h)}{\cos^3\theta} \mathrm{d}\theta
    \end{align*} \right.\\
    &= \int_1^{\tan^2\theta_h A(\phi_h) + 1} \frac{1}{u^2} \mathrm{d}u \\
    &= \Big[-\frac{1}{u}\Big]_1^{\tan^2\theta_h A(\phi_h) + 1} \\
    &= 1 - \frac{1}{\tan^2\theta_h A(\phi_h) + 1} \\
    &= 1 - \frac{1}{\tan^2\theta_h \left(\frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y}\right) + 1} \\
\end{align*}$$

Inverting this function and solving for $\theta_h$ we get:

$$\begin{align*}
\theta_h 
    &= \mathrm{P}_{\theta|\phi}^{-1}(\xi_2, \phi_h) \\
    &= \arctan \sqrt{\frac{\xi_2}{A(\phi_h)(1 - \xi_2)}} \\
    &= \arctan \sqrt{\frac{1}{\frac{\cos^2\phi_h}{\alpha^2_x}+\frac{\sin^2\phi_h}{\alpha^2_y}} \frac{\xi_2}{1 - \xi_2}} \\
\end{align*}$$

### Generalized Trowbridge-Reitz

There's also a generalized form where the power of two is replaced by an additional parameter $\gamma$ (gamma):

$D_\text{GTR}(\omega_h) = \frac{(\gamma - 1)(\alpha^2 - 1)}{\pi(1 - \alpha^{2(1-\gamma)})}  \Big(\frac{1}{(1+(\alpha^2-1)\cos^2\theta_h}\Big)^\gamma$

Indeed, for $\gamma=2$, this becomes equivalent to Trowbridge-Reitz, hence the $D_\text{GTR2}(\omega_h)$ above

## Geometric Shadowing

### Cook-Torrance

$G_\text{CT}(\omega_l,\omega_v,\omega_h) = \min \Big( 1, \frac{2 (\omega_n \cdot \omega_h)(\omega_n \cdot \omega_v) }{ \omega_v \cdot \omega_h } , \frac{2 (\omega_n \cdot \omega_h)(\omega_n \cdot \omega_l) }{ \omega_v \cdot \omega_h } \Big)$

### Neumann

$G_\text{Neumann}(\omega_l,\omega_v,\omega_h) = \frac{(\omega_n \cdot \omega_l) (\omega_n \cdot \omega_v)}{\max\big((\omega_n \cdot \omega_l), (\omega_n \cdot \omega_v)\big)}$ [Neu1999]

### Smith

Smith has two common forms [Heitz2014]:
-   Separable Masking and Shadowing $G_{2,\text{Smith,separable}}$, popularized by [WLMT2007]
    $$\begin{align*}
    G_{2,\text{Smith,separable}}(\omega_o, \omega_i, \omega_h)
        &= G_{1,\text{Smith}}(\omega_o, \omega_h) G_{1,\text{Smith}}(\omega_i, \omega_h) \\
        &= \frac{\chi^+ (\omega_o \cdot \omega_h)}{1 + \Lambda(\omega_o)} \frac{\chi^+ (\omega_i \cdot \omega_h)}{1 + \Lambda(\omega_i)}
    \end{align*}$$

-   Height-Correlated Masking and Shadowing $G_{2,\text{Smith,height-correlated}}$
    $$G_{2,\text{Smith,height-correlated}}(\omega_o, \omega_i, \omega_h) 
        = \frac{\chi^+ (\omega_o \cdot \omega_h) \chi^+ (\omega_i \cdot \omega_h)}{1 + \Lambda(\omega_o) + \Lambda(\omega_i)}$$

$\Lambda(\omega)$ depends on the Microfacet Distribution Function being used, and can be derived via:

$$\frac{1}{1+\Lambda(\omega)} = \frac{\cos \theta}{\int_\Omega (\omega \cdot \omega_h) D(\omega_h) \mathrm{d} \omega_h}$$

#### Beckmann

With $a = \frac{1}{\alpha \tan \theta}$:

$$\begin{align*}
\Lambda_\text{Beckmann}(\omega)
    &= \frac{\mathrm{erf}(a) - 1}{2} + \frac{1}{2a \sqrt{\pi}} \exp(-a^2) \\
    &\approx \begin{cases}
        \frac{1 - 1.259 a + 0.396 a^2}{3.535 a + 2.181 a^2} & \text{if } a < 1.6 \\
        1 & \text{otherwise}
    \end{cases}
\end{align*}$$

In the anisotropic case we have [Heitz2014]:

$$\begin{align*}
a &= \frac{1}{\sqrt{\alpha_x^2 \cos^2 \phi + \alpha_y^2 \sin^2 \phi} \tan \theta} \\
  &= \frac{h_z}{\sqrt{\alpha_x^2 h_x^2 + \alpha_y^2 h_y^2}}
\end{align*}$$

#### Trowbridge-Reitz

#### Blinn

There's no closed form solution. [WLMT2007] suggests to use the same one as for Beckmann.

## BRDFs

The general microfacet BRDF as formulated by Torrance-Sparrow is:

$$f_r(\omega_l,\omega_v) = \frac{D(\omega_h) \mathrm{F}(\omega_v,\omega_h) \mathrm{G}(\omega_l,\omega_v,\omega_h)} {4 (\omega_n \cdot \omega_l) (\omega_n \cdot \omega_v)}$$

$$\begin{align*}
\mathrm{p}({\omega_l}) 
    &= \frac{\mathrm{p}(\omega_h)} {4 (\omega_l \cdot \omega_h)} \\
    &= \frac{D(\omega_h) \cos \theta_h} {4 (\omega_l \cdot \omega_h)}
\end{align*}$$

### Ashikmin Shirley

Here's the specular component in full, with $\omega_l = \textbf{k}_1$, $\omega_v = \textbf{k}_2$ [AS2000]:

$$f_{r,\text{AS}}(\omega_l,\omega_v) = \frac{\sqrt{(n_u+1)(n_v+1)}}{8\pi} \frac{(\omega_n \cdot \omega_h)^{n_u\cos^2\phi+n_v\sin^2\phi}}{(\omega_h \cdot \omega_v) \max [ (\omega_n \cdot \omega_l), (\omega_n \cdot \omega_v) ]} \mathrm{F}_\text{schlick}(\omega_h \cdot \omega_v)$$

With $D_\text{Blinn,aniso}(\omega_h)$ this becomes:

$f_{r,\text{AS}}(\omega_l,\omega_v) = \frac{D_\text{Blinn,aniso}(\omega_h) \mathrm{F}_\text{schlick}(\omega_h \cdot \omega_v)}{4(\omega_h \cdot \omega_v) \max\big((\omega_n \cdot \omega_l), (\omega_n \cdot \omega_v)\big)}$

This formulation corresponds to the `FresnelBlend` BRDF in [PBRT2017], except PBRT uses $D_\text{TR,aniso}(\omega_h)$.

To fit this in the Torrance-Sparrow model, we need:

$G_\text{AS}(\omega_l,\omega_v,\omega_h)
= \frac{(\omega_n \cdot \omega_l) (\omega_n \cdot \omega_v)}{(\omega_h \cdot \omega_v) \max\big((\omega_n \cdot \omega_l), (\omega_n \cdot \omega_v)\big)}
= \frac{1}{(\omega_h \cdot \omega_v)} G_\text{Neumann}(\omega_l,\omega_v,\omega_h)$

So what's the mystery factor $\frac{1}{(\omega_h \cdot \omega_v)}$ ?

### Cook-Torrance

The BRDF in [CT] uses $D_\text{Beckmann,aniso}(\omega_h)$ and $G_\text{CT}(\omega_l,\omega_v,\omega_h)$:

$f_{r,CT}(\omega_l,\omega_v) = \frac{D_\text{Beckmann,aniso}(\omega_h) F(\omega_v,\omega_h) G_\text{CT}(\omega_l,\omega_v,\omega_h)} {4 (\omega_n \cdot \omega_l) (\omega_n \cdot \omega_v)}$

### Walter

The BRDF in [WLMT2007] uses $D_\text{TR,aniso}(\omega_h)$ and the separable Smith $G(\omega_l,\omega_v,\omega_h)$:

$f_{r,WLMT}(\omega_l,\omega_v) = \frac{D_\text{TR,aniso}(\omega_h) F(\omega_v,\omega_h) G1_{TR,aniso}(\omega_l,\omega_h) G1_{TR,aniso}(\omega_v,\omega_h)} {4 (\omega_n \cdot \omega_l) (\omega_n \cdot \omega_v)}$

### PBRT MicrofacetReflection

The `MicrofacetReflection` BRDF uses the height correlated Smith $G(\omega_l,\omega_v,\omega_h)$ and can use both $D_\text{Beckmann,aniso}(\omega_h)$ and $D_\text{TR,aniso}(\omega_h)$, but only $D_\text{TR,aniso}(\omega_h)$ is used.

$f_r(\omega_l,\omega_v) = \frac{D_\text{TR,aniso}(\omega_h) F(\omega_v,\omega_h) \frac{1}{1 + \Lambda_\text{TR,aniso}(\omega_l) + \Lambda_\text{TR,aniso}(\omega_v)}} {4 (\omega_n \cdot \omega_l) (\omega_n \cdot \omega_v)}$
